In [ ]:
import numpy as np
import sounddevice as sd
import librosa
import tflite_runtime.interpreter as tflite
import time
import logging

# ----------------------------
# Logging
# ----------------------------
logging.basicConfig(level=logging.INFO)

# ----------------------------
# Parameters
# ----------------------------
MODEL_PATH = "model.tflite"     # Pretrained model path
SAMPLE_RATE = 16000             # Must match your model
DURATION = 2                    # Record for 2 seconds
N_MFCC = 40                     # Must match model input

# Class labels for model output
CLASSES = [
    "dog_bark", "children_playing", "air_conditioner", "street_music",
    "engine_idling", "jackhammer", "drilling", "siren", "car_horn", "gun_shot"
]

# ----------------------------
# Load TFLite model
# ----------------------------
interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

logging.info("Model loaded successfully!")

# ----------------------------
# Record audio
# ----------------------------
def record_audio():
    logging.info(" Recording started...")
    audio = sd.rec(int(SAMPLE_RATE * DURATION), samplerate=SAMPLE_RATE, channels=1, dtype='float32')
    sd.wait()
    logging.info(" Recording finished")
    return np.squeeze(audio)

# ----------------------------
# Extract MFCC features
# ----------------------------
def extract_mfcc(audio):
    mfcc = librosa.feature.mfcc(y=audio, sr=SAMPLE_RATE, n_mfcc=N_MFCC)
    mfcc = np.mean(mfcc, axis=1)
    mfcc = mfcc.reshape(1, N_MFCC).astype(np.float32)
    logging.info(f"MFCC shape: {mfcc.shape}")
    return mfcc

# ----------------------------
# Run inference
# ----------------------------
def predict(mfcc):
    interpreter.set_tensor(input_details[0]['index'], mfcc)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    prediction = np.argmax(output)
    confidence = np.max(output)
    return CLASSES[prediction], confidence


if __name__ == "__main__":
    logging.info("Starting real-time audio classification...")
    time.sleep(2)

    while True:
        input("➡ Press Enter to classify audio...")

        audio = record_audio()
        mfcc = extract_mfcc(audio)
        label, conf = predict(mfcc)

        print(f"\n Prediction: {label}    Confidence: {conf:.2f}\n")
